# Hydrology skill - Vistula river

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/DHI/modelskill-release-webinar/blob/main/Hydrology.ipynb)


In [ ]:
!git clone https://github.com/DHI/modelskill-release-webinar

In [ ]:
!cp -r modelskill-release-webinar/examples/ examples/

In [ ]:
!pip install modelskill mapclassify

In [ ]:
import pandas as pd
import mikeio
import modelskill as ms

In [ ]:
idx = 6458010

ds = mikeio.read(f"examples/hydrology/data/{idx}.dfs0")
ds


In [ ]:
cmp = ms.from_matched("examples/hydrology/data/6458010.dfs0", obs_item=0, mod_items=[1,2], name="TCEW")
cmp

In [ ]:
cmp.plot.timeseries(backend="plotly")

In [ ]:
cmp.plot.scatter();
#cmp.plot.scatter(backend="plotly", skill_table=True);

In [ ]:
cmp.plot.kde();

In [ ]:
cmp.plot.qq();

In [ ]:
cmp.skill()

In [ ]:
cmp.skill(metrics=["kge","cc"])

<https://dhi.github.io/modelskill/api/metrics/>

In [ ]:
from modelskill import options

options.metrics.list = ["kge","willmott","spearmanr"]

In [ ]:
cmp.skill()

In [ ]:
cmp.skill(by="freq:A").kge.plot.line()


In [ ]:
cmp.skill(metrics=["kge"], by="dt:month").kge.plot.bar();

## Many locations

In [ ]:
stations = pd.read_csv("examples/hydrology/stations.csv").sort_values("weight", ascending=False)
stations.head()

In [ ]:
cmps = []
for _, row in stations.iterrows():
    cmp = ms.from_matched(f"examples/hydrology/data/{row.id}.dfs0",x=row.lon,y=row.lat,obs_item=0, mod_items=[1,2], name=row['name'])
    cmp.data.attrs["gtype"] = "point" # temp work-around
    cmps.append(cmp)

In [ ]:
cc = ms.ComparerCollection(cmps)
cc

In [ ]:
weights = dict(zip(stations.name, stations.weight))
weights

In [ ]:
cc.score(metric="kge", weights='equal')

In [ ]:
cc.score(metric="kge", weights=weights)

In [ ]:
cc.skill(metrics="kge")["kge"].plot.barh()

In [ ]:
gdf = (cc.sel(model="sim1")
        .skill(metrics="kge")
        .to_geodataframe()
)
gdf

In [ ]:
gdf.explore("kge", marker_kwds={"radius":10}, tiles="CartoDB positron", cmap="RdYlGn")

In [ ]:
import plotly.express as px

sdf = (cc.skill(metrics="kge")
         .to_dataframe()
         .sort_values("kge", ascending=True)
         .reset_index()
         .replace({"sim2": "HiFi", "sim1": "LoFi"})
)
sdf.head()

In [ ]:
fig = px.scatter(sdf, x="kge", y="observation",color='model', symbol='model', width=800)
fig.update_layout(yaxis_title="", xaxis_title="KGE (-)")